# [SOURCE] BINOSNP ANALYSIS

In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
import os

import sys
sys.path.append("./scripts/modules")

from benchmarking_definitions import *
from regions import *

In [2]:
tagsRv = pd.read_csv("./data/source/FINAL_tag_list.csv", names=["tag"])["tag"].values
tagsL1234 = pd.read_csv("./data/source/FINAL_tag_list_L1234.csv", names=["tag"]).tag.values

len(tagsRv), len(tagsL1234)

(2500, 1000)

In [3]:
Rv_binosnp_source = "/n/data1/hms/dbmi/farhat/shandu/projects/mixed_calls/benchmarking/comprehensive/ISS_pipeline_outputs/binosnp_run2/output"
L1234_binosnp_source = "/n/data1/hms/dbmi/farhat/shandu/projects/mixed_calls/benchmarking/comprehensive/ISSL123_pipeline_outputs/binosnp_run2/output"

In [4]:
Rv_mutant_summary_source = "/n/data1/hms/dbmi/farhat/shandu/projects/mixed_calls/benchmarking/comprehensive/ISS_mutant_summaries/output"
L1234_mutant_summary_source = "/n/data1/hms/dbmi/farhat/shandu/projects/mixed_calls/benchmarking/comprehensive/ISSL123_mutant_summaries/output"

In [5]:
Rv_output_dir = "/n/scratch/users/s/sm624/ISS_H37Rv_binosnp_summaries"
L1234_output_dir = "/n/scratch/users/s/sm624/ISS_L1234_binosnp_summaries"

In [6]:
Rv_FP_label_source_dir = "/n/scratch/users/s/sm624/ISS_H37Rv_label_FP/output"
L1234_FP_label_source_dir = "/n/scratch/users/s/sm624/ISS_L1234_label_FP/output"

# Variant filtering utility

## Merge binosnp summary tables with overall mutant summary tables

### H37Rv

In [57]:
count = 0

for tag in tagsRv:

    if count % 10 == 0:
        print(count, end=" ")
    count += 1

    binosnp_summary_df = pd.read_csv("{}/{}.binosnp.final.csv".format(Rv_binosnp_source, tag))
    mutant_summary_df = pd.read_csv("{}/{}.csv".format(Rv_mutant_summary_source, tag))

    df1 = binosnp_summary_df[["POS", "REF", "ALT", "BS_TYPE", "BS_found", "BS_AF"]].copy(deep=True)
    df2 = mutant_summary_df.copy(deep=True)

    df1.set_index(["POS", "REF", "ALT"], inplace=True)
    df2.set_index(["POS", "REF", "ALT"], inplace=True)

    merged_df = df1.join(df2, how='outer')
    
    merged_df["BS_found"] = merged_df["BS_found"].fillna(0)

    merged_df.to_csv("{}/{}.csv".format(Rv_output_dir, tag), index=True)


0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 1910 1920 1930 1940 1950 1960 1970 1980 1990 2000 2010 2020 2030 2040 2050 2060 2070 2080 2090 2100 2110 2120 2130 2140 2150 2160 2170 2180 2190 2200 2210 2

### L1234

In [68]:
count = 0

for tag in tagsL1234:

    if count % 10 == 0:
        print(count, end=" ")
    count += 1

    binosnp_summary_df = pd.read_csv("{}/{}.binosnp.final.csv".format(L1234_binosnp_source, tag))
    mutant_summary_df = pd.read_csv("{}/{}.final.csv".format(L1234_mutant_summary_source, tag))

    df1 = binosnp_summary_df[["POS", "REF", "ALT", "BS_TYPE", "BS_found", "BS_AF"]].copy(deep=True)
    df2 = mutant_summary_df.copy(deep=True)

    df1.set_index(["POS", "REF", "ALT"], inplace=True)
    df2.set_index(["POS", "REF", "ALT"], inplace=True)

    merged_df = df1.join(df2, how='outer')
    
    merged_df["BS_found"] = merged_df["BS_found"].fillna(0)

    merged_df.to_csv("{}/{}.csv".format(L1234_output_dir, tag), index=True)


0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 

## Determine number of FP detected by any one variant caller and the number of FP detected by BinoSNP for each strain

## Overall (keep track of depth and genome)

In [99]:
found_cols = ["{}_found".format(t) for t in tools]
AF_cols = ["{}_AF".format(t) for t in tools]

In [118]:
binosnp_FP_analysis = pd.DataFrame(columns=["genome", "tag", "depth", "num_FP_one_other_tool", "num_FP_BS"])
df_i = 0

count = 0
for tag in tagsRv:

    genome = "H37Rv"

    depth = int(tag.split("_")[-2])

    if count % 10 == 0:
        print(count, end=" ")
    count += 1

    final_summary_df = pd.read_csv("{}/{}.csv".format(Rv_output_dir, tag))

    # find FP that are detected by at least one other tool
    FP_df = final_summary_df[(final_summary_df.GT == 0) & (final_summary_df[["{}_found".format(t) for t in tools]].sum(axis=1) >= 1)]
    
    # exclude FP if there is no tool that detects it at AF > 1%
    found_vals = FP_df[found_cols].values
    AF_vals = FP_df[AF_cols].values
    
    valid_mask = (found_vals == 1) & (AF_vals != 0.01)
    valid_counts = valid_mask.sum(axis=1)
    
    FP_excludeAF1_df = FP_df[valid_counts >= 1]
        
    num_FP_one_other_tool = FP_excludeAF1_df.shape[0]

    BS_FP_df = FP_excludeAF1_df[FP_excludeAF1_df.BS_found == 1]
    num_FP_BS = BS_FP_df.shape[0]

    binosnp_FP_analysis.loc[df_i] = [genome, tag, depth, num_FP_one_other_tool, num_FP_BS]
    df_i += 1

for tag in tagsL1234:

    genome = tag.split("_")[1]

    depth = int(tag.split("_")[-2])

    if count % 10 == 0:
        print(count, end=" ")
    count += 1

    final_summary_df = pd.read_csv("{}/{}.csv".format(L1234_output_dir, tag))

    # find FP that are detected by at least one other tool
    FP_df = final_summary_df[(final_summary_df.L_GT == 0) & (final_summary_df.GT == 0) & (final_summary_df[["{}_found".format(t) for t in tools]].sum(axis=1) >= 1)]
    
    # exclude FP if there is no tool that detects it at AF > 1%
    found_vals = FP_df[found_cols].values
    AF_vals = FP_df[AF_cols].values
    
    valid_mask = (found_vals == 1) & (AF_vals != 0.01)
    valid_counts = valid_mask.sum(axis=1)
    
    FP_excludeAF1_df = FP_df[valid_counts >= 1]
        
    num_FP_one_other_tool = FP_excludeAF1_df.shape[0]

    BS_FP_df = FP_excludeAF1_df[FP_excludeAF1_df.BS_found == 1]
    num_FP_BS = BS_FP_df.shape[0]

    binosnp_FP_analysis.loc[df_i] = [genome, tag, depth, num_FP_one_other_tool, num_FP_BS]
    df_i += 1
    


0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 1910 1920 1930 1940 1950 1960 1970 1980 1990 2000 2010 2020 2030 2040 2050 2060 2070 2080 2090 2100 2110 2120 2130 2140 2150 2160 2170 2180 2190 2200 2210 2

In [119]:
denominator = binosnp_FP_analysis["num_FP_one_other_tool"].replace(0, np.nan)
binosnp_FP_analysis["percentage_FP_BS_found"] = binosnp_FP_analysis["num_FP_BS"].div(denominator, axis=0)*100

In [120]:
binosnp_FP_analysis

,genome,tag,depth,num_FP_one_other_tool,num_FP_BS,percentage_FP_BS_found
0,H37Rv,sim1_h37rv_mutant_1_50_0.01,50,328,314,95.731707
1,H37Rv,sim1_h37rv_mutant_2_50_0.01,50,293,279,95.221843
2,H37Rv,sim1_h37rv_mutant_3_50_0.01,50,314,299,95.222930
3,H37Rv,sim1_h37rv_mutant_4_50_0.01,50,306,291,95.098039
4,H37Rv,sim1_h37rv_mutant_5_50_0.01,50,324,310,95.679012
...,...,...,...,...,...,...
3495,L4,sim1_L4_mutant_700_0.5,700,1587,986,62.129805
3496,L4,sim2_L4_mutant_700_0.5,700,1558,988,63.414634
3497,L4,sim3_L4_mutant_700_0.5,700,1623,946,58.287123
3498,L4,sim4_L4_mutant_700_0.5,700,1699,988,58.151854


In [121]:
binosnp_FP_analysis.to_csv("./data/binosnp_analysis/ISS_binosnp_FP_analysis.csv", index=False)

## By mutation region

Use FP labels determined in [C].

In [124]:
binosnp_FP_by_region_analysis = pd.DataFrame(columns=["genome", "tag", "depth", "region", "num_FP_one_other_tool", "num_FP_BS"])
df_i = 0

count = 0
for tag in tagsRv:

    genome = "H37Rv"

    depth = int(tag.split("_")[-2])

    if count % 10 == 0:
        print(count, end=" ")
    count += 1

    one_other_tool_tag_region_FP_counts = {r:0 for r in regions_other}
    BS_tag_region_FP_counts = {r:0 for r in regions_other}

    final_summary_df = pd.read_csv("{}/{}.csv".format(Rv_output_dir, tag))

    with open("{}/{}.pkl".format(Rv_FP_label_source_dir, tag), "rb") as in_f:
        tag_FP_labels = pkl.load(in_f)
        flattened_FP_labels = {}
        for tag_tool_FP_labels in tag_FP_labels.values():
            flattened_FP_labels.update(tag_tool_FP_labels)
            #fine to overwrite previous keys because the region label of a POS will always be the same

    FP_df = final_summary_df[final_summary_df.GT == 0]
    
    # exclude FP if there is no tool that detects it at AF > 1%
    found_vals = FP_df[found_cols].values
    AF_vals = FP_df[AF_cols].values
    
    valid_mask = (found_vals == 1) & (AF_vals != 0.01)
    valid_counts = valid_mask.sum(axis=1)
    
    FP_excludeAF1_df = FP_df[valid_counts >= 1]
    
    for i in FP_excludeAF1_df.index:

        POS = FP_excludeAF1_df.loc[i, "POS"]
        pos_region = flattened_FP_labels[POS][0]

        # find FP that are detected by at least one other tool
        tools_found = FP_excludeAF1_df.loc[i, ["{}_found".format(t) for t in tools]]
        
        if np.any(tools_found):
            # one other variant caller found this FP
            one_other_tool_tag_region_FP_counts[pos_region] += 1
            
            if FP_excludeAF1_df.loc[i, "BS_found"] == 1:
                # BS also found this FP
                BS_tag_region_FP_counts[pos_region] += 1

    for region in regions_other:
        binosnp_FP_by_region_analysis.loc[df_i] = [genome, tag, depth, region, one_other_tool_tag_region_FP_counts[region], BS_tag_region_FP_counts[region]]
        df_i += 1

for tag in tagsL1234:

    genome = "L1234"

    depth = int(tag.split("_")[-2])

    if count % 10 == 0:
        print(count, end=" ")
    count += 1

    final_summary_df = pd.read_csv("{}/{}.csv".format(L1234_output_dir, tag))

    with open("{}/{}.pkl".format(L1234_FP_label_source_dir, tag), "rb") as in_f:
        tag_FP_labels = pkl.load(in_f)
        flattened_FP_labels = {}
        for tag_tool_FP_labels in tag_FP_labels.values():
            flattened_FP_labels.update(tag_tool_FP_labels)
            #fine to overwrite previous keys because the region label of a POS will always be the same

    FP_df = final_summary_df[(final_summary_df.L_GT == 0) & (final_summary_df.GT == 0)]

    # exclude FP if there is no tool that detects it at AF > 1%
    found_vals = FP_df[found_cols].values
    AF_vals = FP_df[AF_cols].values
    
    valid_mask = (found_vals == 1) & (AF_vals != 0.01)
    valid_counts = valid_mask.sum(axis=1)
    
    FP_excludeAF1_df = FP_df[valid_counts >= 1]

    for i in FP_excludeAF1_df.index:

        POS = FP_excludeAF1_df.loc[i, "POS"]
        pos_region = flattened_FP_labels[POS][0]

        # find FP that are detected by at least one other tool
        tools_found = FP_excludeAF1_df.loc[i, ["{}_found".format(t) for t in tools]]
        
        if np.any(tools_found):
            # one other variant caller found this FP
            one_other_tool_tag_region_FP_counts[pos_region] += 1
            
            if FP_excludeAF1_df.loc[i, "BS_found"] == 1:
                # BS also found this FP
                BS_tag_region_FP_counts[pos_region] += 1

    for region in regions_other:
        binosnp_FP_by_region_analysis.loc[df_i] = [genome, tag, depth, region, one_other_tool_tag_region_FP_counts[region], BS_tag_region_FP_counts[region]]
        df_i += 1
    


0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 1910 1920 1930 1940 1950 1960 1970 1980 1990 2000 2010 2020 2030 2040 2050 2060 2070 2080 2090 2100 2110 2120 2130 2140 2150 2160 2170 2180 2190 2200 2210 2

In [125]:
denominator = binosnp_FP_by_region_analysis["num_FP_one_other_tool"].replace(0, np.nan)
binosnp_FP_by_region_analysis["percentage_FP_BS_found"] = binosnp_FP_by_region_analysis["num_FP_BS"].div(denominator, axis=0)*100

In [126]:
binosnp_FP_by_region_analysis

,genome,tag,depth,region,num_FP_one_other_tool,num_FP_BS,percentage_FP_BS_found
0,H37Rv,sim1_h37rv_mutant_1_50_0.01,50,DR,1,1,100.000000
1,H37Rv,sim1_h37rv_mutant_1_50_0.01,50,HT,0,0,NaN
2,H37Rv,sim1_h37rv_mutant_1_50_0.01,50,LM,24,22,91.666667
3,H37Rv,sim1_h37rv_mutant_1_50_0.01,50,other,303,291,96.039604
4,H37Rv,sim1_h37rv_mutant_2_50_0.01,50,DR,3,3,100.000000
...,...,...,...,...,...,...,...
13995,L1234,sim4_L4_mutant_700_0.5,700,other,399030,273048,68.427938
13996,L1234,sim5_L4_mutant_700_0.5,700,DR,2951,2444,82.819383
13997,L1234,sim5_L4_mutant_700_0.5,700,HT,881,36,4.086266
13998,L1234,sim5_L4_mutant_700_0.5,700,LM,1519364,953328,62.745201


In [127]:
binosnp_FP_by_region_analysis.to_csv("./data/binosnp_analysis/ISS_binosnp_FP_by_region_analysis.csv", index=False)